# <center>This `.ipynb` file contains the code for computing the SSIM metric</center>

### 1. Import the required libraries

In [1]:
import os
import numpy as np
from skimage.metrics import structural_similarity as ssim
from skimage.io import imread
from tqdm import tqdm
from multiprocessing import Pool

### 2. Custom function for calculating the `SSIM` metric

In [2]:
def calculate_ssim(img1, img2):
    return ssim(img1, img2, win_size=3, multichannel=True)

### 3. Custom function for loading images

In [3]:
def load_image(filepath):
    return imread(filepath)

### 4. Prepare to calculate the `SSIM` metric

In [4]:
def parallel_ssim(real_generated_pair):
    
    real_path, generated_path = real_generated_pair
    real_img = load_image(real_path)
    generated_img = load_image(generated_path)
    
    return calculate_ssim(real_img, generated_img)

In [5]:
def batch_process_ssim_parallel(real_dir, gen_dir, batch_size=32):
    
    real_images = sorted([os.path.join(real_dir, f) for f in os.listdir(real_dir) if f.endswith(('jpg'))])
    gen_images = sorted([os.path.join(gen_dir, f) for f in os.listdir(gen_dir) if f.endswith(('jpg'))])
    
    assert len(real_images) == len(gen_images), 'Mismatch in the number of images between real and generated directories'
    
    pairs = list(zip(real_images, gen_images))
    
    with Pool() as pool:
        ssim_values = list(tqdm(pool.imap(parallel_ssim, pairs), total=len(pairs)))
        
    return ssim_values

### 6. Show the result

In [6]:
select_batch_size = 50

real_dir = '../org'
print('Total images in real_dir:', len(next(os.walk(real_dir))[2]))

generated_dir =  'gen_14'
print('Total images in generated_dir:', len(next(os.walk(generated_dir))[2]))

ssim_scores = batch_process_ssim_parallel(real_dir, generated_dir, batch_size=select_batch_size)
print(f'Mean SSIM: {np.mean(ssim_scores):.4f}')

Total images in real_dir: 10000
Total images in generated_dir: 10000


100%|████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:43<00:00, 232.53it/s]

Mean SSIM: 0.7988
